In [ ]:
import torch.nn as nn
import torch.nn.functioanl as F

class MultipleClassificationModel(nn.Module):
    def __init__(self, num_classes):
        """
        모델의 초기화 함수, 모델에 사용할 각 계층을 정의
        num_classes : 분류할 클래스의 수
        """
        super().__init__()
        
        #  첫 번째 Convolution Block
        # Conv2d :  이미지에서 특성을 추출하는 층
        # ( 입력 채널 : 3, 출력 채널 : 32, 커널 크기 : 3 x 3 )  / 커널(= 필터, 마스크) : 이미지로부터 원하는 특징만 추출하는 것
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) # RGB 이미지 ( 채널 3 ) 을 입력받고, 32개의 특성 맵을 추출
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 32 개 특성 맵을 입력받고, 64개로 확장
        self.pool = nn.MaxPoo2d(2, 2) # Max Pooling : 특성 맵의 크기를 절반으로 줄임 ( => 계산량 감소 )
        
        # 두 번째 Convolution Block
        self.conv3 = nn.Conv2d(64, 128, kerner_size=3, padding=1) # 64개의 특성 맵을 받아 128개로 확장
        self.conv4 = nn.Conv2d(128, 256, kerner_size=3, padding=1) # 128개의 특성 맵을 받아 256개로 확장
        
        # Fully Connected Layers ( FC ) : 마지막에 나온 특성 맵을 1차원 벡터로 펼쳐서 분류 작업을 시행
        self.fc1 = nn.Linear(256 * 8 * 8, 1024) # 마지막 특성 맵을 1024개의 뉴런으로 변환
        self.fc2 = nn.Linear(1024, num_classes) # 1024개의 뉴런을 받아 최종 클래스 개수에 맞는 출력값을 생성'
    
    def forward(self, x):
        """
        입력 데이터 x를 모델을 통해 순차적으로 전달하며 예측을 생성
        """
        
        #  첫 번째 Convolution Block
        x = F.relu(self.conv1(x)) # 첫 번째 Conv2d + ReLU 활성화 함수
        x = self.pool(F.relu(self.conv2(x))) # 두 번째 Conv2d + ReLU + Max pooling
        
        x = F.relu(self.conv3(x)) # 세 번째 Conv2d + ReLU 활성화 함수
        x = self.pool(F.relu(self.conv4(x))) # 네 번째 Conv2d + ReLU + Max pooling
        
        # 전역 평균 풀링 (  Global Average Pooling ) : 특성 맵을 1차원 벡터로 평탄화
        x = x.view(-1, 256 * 8 * 8) # 특성 맵을 1차원 벡터로 펼침
        
        # Fully Connected Layers ( FC ) : 최종 분류 결과를 출력
        x = F.relu(self.fc1(x)) # 첫 번째 FC + ReLU 활성화 함수
        x = self.fc2(x) # 두 번째 FC : 최송 클래스에 대한 예측 결과
        return x